In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.sklearn

# spacy for lemmatization
import spacy

# Plotting tools
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import nltk

# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

# Plotting tools
import matplotlib.pyplot as plt
%matplotlib inline

D:\Software\Anaconda3\envs\pytorch\lib\site-packages\past\builtins\misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
D:\Software\Anaconda3\envs\pytorch\lib\site-packages\joblib\backports.py:22: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  import distutils  # noqa
D:\Software\Anaconda3\envs\pytorch\lib\site-packages\sklearn\utils\multiclass.py:14: DeprecationWarning: Please use `spmatrix` from the `scipy.sparse` namespace, the `scipy.sparse.base` namespace is deprecated.
  from scipy.sparse.base import spmatrix
D:\Software\Anaconda3\envs\pytorch\lib\site-packages\sklearn\utils\optimize.py:18: DeprecationWarning: Please use `line_search_wolfe2` from the `scipy.optimize` namespace, the `scipy.optimize.linesearch` namespace is deprec

In [3]:
df = pd.read_csv('./dataset/NMSendataset_wordtoken_lda_2023.csv', error_bad_lines=False);
df

,_id,recommend,found_helpful,found_funny,Column1,time,reply_count,content,hrs_on_second,hrs_at_reviwer_time,reviwer_num,language,sent_content,preprocessed_reviews_sec,wordtoken,correctmapping,preprocessed_reviews_lda
0,5f75ad269136b25c3c8b50f8,True,1,0,9,2023/2/29,0,\r\n\t\t\t\t\t\t\t\tthis good\t\t\t\t\t\t\t,0.5,37.9,15,en,\r\n\t\t\t\t\t\t\t\tthis good,this good,['good'],"['this', 'good']",good
1,5f771cbbee89a0be76bd8c80,True,1,0,9,2023/2/29,0,\r\n\t\t\t\t\t\t\t\tWhat a difference a few ye...,19.4,352.6,16,en,\r\n\t\t\t\t\t\t\t\tWhat a difference a few ye...,what a difference a few year make .,"['difference', 'year']","['what', 'difference', 'few', 'year', 'make']",difference year make
2,5f771cbbee89a0be76bd8c80,True,1,0,9,2023/2/29,0,\r\n\t\t\t\t\t\t\t\tWhat a difference a few ye...,19.4,352.6,16,en,"From its messy origins, No Man's Sky has grown...",from it messy origin: no man be sky have grown...,"['messy', 'origin', 'no_man_sky', 'grown', 'de...","['from', 'it', 'messy', 'origin', 'no', 'man',...",messy origin no man sky grown deeper polished ...
3,5f771cbbee89a0be76bd8c80,True,1,0,9,2023/2/29,0,\r\n\t\t\t\t\t\t\t\tWhat a difference a few ye...,19.4,352.6,16,en,"The vast universe is enticing, and there's ple...",the vast universe be entice: and there be plen...,"['vast', 'universe', 'entice', 'plenty', 'most...","['the', 'vast', 'universe', 'be', 'entice', 'a...",vast universe entice plenty mostly either expl...
4,5f771cbbee89a0be76bd8c80,True,1,0,9,2023/2/29,0,\r\n\t\t\t\t\t\t\t\tWhat a difference a few ye...,19.4,352.6,16,en,"And with proper multiplayer now introduced, it...",and with proper multiplayer now introduce: it ...,"['proper', 'multiplayer', 'introduce', 'lonely...","['and', 'with', 'proper', 'multiplayer', 'now'...",proper multiplayer introduce lonely experience...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
476338,64abdb8a38d28815033e7a39,False,0,0,5,12/08/2016,1,\r\n\t\t\t\t\t\t\t\tThis isn't a PC game. It'...,0.0,44.9,37,en,"If the game was $20 I'd keep it, but definitel...",if the game be $ 20 i would keep it: but defin...,"['20', 'would', 'definitely', 'worth_60', 'ref...","['if', 'the', 'game', 'be', '20', 'would', 'ke...",20 would keep definitely worth 60 refund time
476339,64abf8e1b4d8ae53485d2c42,True,0,0,5,12/08/2016,0,\r\n\t\t\t\t\t\t\t\tIt's pretty interesting\t\...,0.0,23.2,42,en,\r\n\t\t\t\t\t\t\t\tIt's pretty interesting,\r\n it be pretty interest,"['pretty', 'interest']","['it', 'be', 'pretty', 'interest']",pretty interest
476340,64ac11efb4d8ae53485d2c6b,False,0,0,5,12/08/2016,2,\r\n\t\t\t\t\t\t\t\tI dislike the fact that al...,0.0,53.4,17,en,\r\n\t\t\t\t\t\t\t\tI dislike the fact that al...,\r\n i dislike the fact that all the creature ...,"['dislike', 'fact', 'creature', 'promise', 'cr...","['dislike', 'the', 'fact', 'that', 'all', 'the...",dislike fact creature promise creature keep sh...
476341,64ac11efb4d8ae53485d2c6b,False,0,0,5,12/08/2016,2,\r\n\t\t\t\t\t\t\t\tI dislike the fact that al...,0.0,53.4,17,en,Please add the creatures into the game.,please add the creature into the game .,"['please', 'add', 'creature']","['please', 'add', 'the', 'creature', 'into', '...",please add creature


In [4]:
def convert(s):
    s = eval(s)
    s = ' '.join(s)
    return s
df['wordtoken'] = df['wordtoken'].apply(convert)

In [5]:
data = df.ngram.values.tolist()
data[10]

'recommend launch disaster yet tail grab much income possible bail hello_games developer stuck turn everything get true multiplayer virtual_reality time pull trigger developers thanks thing'

In [6]:
tfidf_vectorizer = TfidfVectorizer(
                                 max_df=500, 
                                   max_features=2000,
                                 min_df=0, 
#                                  stop_words='english',
                                   use_idf=True, 
#                                    token_pattern='[a-zA-Z0-9]{3,}', 
#                                    ngram_range=(1,3)
)

%time tfidf_vectorized = tfidf_vectorizer.fit_transform(data)

print(tfidf_vectorized.shape)

CPU times: total: 2.42 s
Wall time: 2.43 s
(97998, 2000)


In [7]:
tfidf_data_vectorized = tfidf_vectorizer.fit_transform(data)

In [8]:
tfidf_vectorizer.get_feature_names()

['aaa_price',
 'aaa_title',
 'absolutly',
 'absurd',
 'abundant',
 'abysmal',
 'abyss',
 'accept',
 'acceptable',
 'access',
 'accidentally',
 'accomplish',
 'accomplishment',
 'accord',
 'account',
 'accurate',
 'achieve',
 'achievement',
 'acid',
 'acknowledge',
 'acquire',
 'act',
 'action_packed',
 'activate',
 'active',
 'actively',
 'activity',
 'actualy',
 'addictive',
 'additional',
 'additionally',
 'address',
 'adjust',
 'admire',
 'admittedly',
 'advance',
 'advanced',
 'advantage',
 'advertisement',
 'advertising',
 'advice',
 'advise',
 'aesthetic',
 'af',
 'affect',
 'afford',
 'afraid',
 'age',
 'aggressive',
 'agree',
 'ahead',
 'aid',
 'aim',
 'aimlessly',
 'air',
 'akin',
 'ala',
 'albeit',
 'algorithm',
 'alien_language',
 'alien_race',
 'alien_specie',
 'alike',
 'alive',
 'allows',
 'alpha',
 'alright',
 'alt_tab',
 'amazing',
 'amazingly',
 'ambient',
 'ambition',
 'ambitious',
 'among',
 'ancient',
 'anger',
 'angry',
 'animal_plant',
 'animals',
 'animation',
 '

In [9]:
tfidf_data_feature_names=tfidf_vectorizer.get_feature_names()

In [10]:
tfidf_data_dense = tfidf_data_vectorized.todense()
# Compute Sparsicity = Percentage of Non-Zero cells
print("tfidf_Sparsicity: ", ((tfidf_data_dense > 0).sum()/tfidf_data_dense.size)*100, "%")
# Since most cells in this matrix will be zero, I am interested in knowing what percentage of cells contain non-zero values.

tfidf_Sparsicity:  0.20869762648217308 %


In [11]:
# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

In [12]:
%%time
lda_model_best = LatentDirichletAllocation(n_components=5,               # Number of topics
                                           learning_decay=0.9,
                                      max_iter=40,               # Max learning iterations
                                      random_state=888,          # Random state
                                      evaluate_every = 5,       # compute perplexity every n iters, default: Don't
                                    perp_tol  = 0.001,
                                      n_jobs = -1,               # Use all available CPUs
                                     )
lda_model_best.fit_transform(tfidf_vectorized)
doc_term_matrix=lda_model_best.fit_transform(tfidf_vectorized)

CPU times: total: 1min 29s
Wall time: 5min


In [13]:
import tmtoolkit
from tmtoolkit.topicmod.evaluate import metric_coherence_gensim
def topic_model_coherence_generator(topic_num_start=2,
                topic_num_end=12,
                step=2,
                norm_corpus='',
                cv_matrix='',
                cv=''):
    norm_corpus_tokens = [doc.split() for doc in norm_corpus]
    # print(norm_corpus_tokens)
    models = []
    coherence_scores = []

    for i in range(topic_num_start, topic_num_end,step):
        print(i)
        lda_model_best = LatentDirichletAllocation(n_components=5,               # Number of topics
                                           learning_decay=0.9,
                                      max_iter=40,               # Max learning iterations
                                      random_state=888,          # Random state
                                      evaluate_every = 5,       # compute perplexity every n iters, default: Don't
                                    perp_tol  = 0.001,
                                      n_jobs = -1,               # Use all available CPUs
                                     )
        lda_model_best.fit_transform(cv_matrix)
        cur_coherence_score = metric_coherence_gensim(
            measure='c_npmi',
            topic_word_distrib=lda_model_best.components_,
            dtm=cv.fit_transform(norm_corpus),
            vocab=np.array(cv.get_feature_names()),
            texts=norm_corpus_tokens)
        models.append(lda_model_best)
        cur_coherence_score = [i for i in cur_coherence_score if str(i)!='nan']
        # print(cur_coherence_score)
        coherence_scores.append(np.mean(cur_coherence_score))
    return models, coherence_scores

In [14]:
end = 6
start = 2
step = 2
import numpy as np
models, coherence_scores = topic_model_coherence_generator(
    start, end,step, norm_corpus=np.array(data), cv=tfidf_vectorizer, cv_matrix=tfidf_vectorized)

2
4


In [15]:
coherence_scores

[-0.23320828045961067, -0.23320828045961067]

In [16]:
def topic_model_coherence_generator_sec(topic_num_start=2,
                topic_num_end=12,
                step=2,
                norm_corpus='',
                cv_matrix='',
                cv=''):
    norm_corpus_tokens = [doc.split() for doc in norm_corpus]
    # print(norm_corpus_tokens)
    models = []
    coherence_scores = []

    for i in range(topic_num_start, topic_num_end,step):
        print(i)
        lda_model_best = LatentDirichletAllocation(n_components=10,               # Number of topics
                                           learning_decay=0.9,
                                      max_iter=40,               # Max learning iterations
                                      random_state=888,          # Random state
                                      evaluate_every = 5,       # compute perplexity every n iters, default: Don't
                                    perp_tol  = 0.001,
                                      n_jobs = -1,               # Use all available CPUs
                                     )
        lda_model_best.fit_transform(cv_matrix)
        cur_coherence_score = metric_coherence_gensim(
            measure='c_v',
            topic_word_distrib=lda_model_best.components_,
            dtm=cv.fit_transform(norm_corpus),
            vocab=np.array(cv.get_feature_names()),
            texts=norm_corpus_tokens)
        models.append(lda_model_best)
        cur_coherence_score = [i for i in cur_coherence_score if str(i)!='nan']
#         print(cur_coherence_score)
        coherence_scores.append(np.mean(cur_coherence_score))
    return models, coherence_scores

In [17]:
models, coherence_scores = topic_model_coherence_generator_sec(
    start, end,step, norm_corpus=np.array(data), cv=tfidf_vectorizer, cv_matrix=tfidf_vectorized)

2
4


In [18]:
coherence_scores

[0.3042787194407589, 0.3042787194407589]

In [19]:
def topic_model_coherence_generator_u_mass(topic_num_start=2,
                topic_num_end=12,
                step=2,
                norm_corpus='',
                cv_matrix='',
                cv=''):
    norm_corpus_tokens = [doc.split() for doc in norm_corpus]
    # print(norm_corpus_tokens)
    models = []
    coherence_scores = []

    for i in range(topic_num_start, topic_num_end,step):
        print(i)
        lda_model_best = LatentDirichletAllocation(n_components=10,               # Number of topics
                                           learning_decay=0.9,
                                      max_iter=40,               # Max learning iterations
                                      random_state=888,          # Random state
                                      evaluate_every = 5,       # compute perplexity every n iters, default: Don't
                                    perp_tol  = 0.001,
                                      n_jobs = -1,               # Use all available CPUs
                                     )
        lda_model_best.fit_transform(cv_matrix)
        cur_coherence_score = metric_coherence_gensim(
            measure='u_mass',
            topic_word_distrib=lda_model_best.components_,
            dtm=cv.fit_transform(norm_corpus),
            vocab=np.array(cv.get_feature_names()),
            texts=norm_corpus_tokens)
        models.append(lda_model_best)
        cur_coherence_score = [i for i in cur_coherence_score if str(i)!='nan']
#         print(cur_coherence_score)
        coherence_scores.append(np.mean(cur_coherence_score))
    return models, coherence_scores

In [20]:
models, coherence_scores = topic_model_coherence_generator_u_mass(
    start, end,step, norm_corpus=np.array(data), cv=tfidf_vectorizer, cv_matrix=tfidf_vectorized)

2
4


In [21]:
coherence_scores

[-7.00336320283414, -7.00336320283414]

In [22]:
def topic_model_coherence_generator_c_uci(topic_num_start=2,
                topic_num_end=12,
                step=2,
                norm_corpus='',
                cv_matrix='',
                cv=''):
    norm_corpus_tokens = [doc.split() for doc in norm_corpus]
    # print(norm_corpus_tokens)
    models = []
    coherence_scores = []

    for i in range(topic_num_start, topic_num_end,step):
        print(i)
        lda_model_best = LatentDirichletAllocation(n_components=10,               # Number of topics
                                           learning_decay=0.9,
                                      max_iter=40,               # Max learning iterations
                                      random_state=888,          # Random state
                                      evaluate_every = 5,       # compute perplexity every n iters, default: Don't
                                    perp_tol  = 0.001,
                                      n_jobs = -1,               # Use all available CPUs
                                     )
        lda_model_best.fit_transform(cv_matrix)
        cur_coherence_score = metric_coherence_gensim(
            measure='c_uci',
            topic_word_distrib=lda_model_best.components_,
            dtm=cv.fit_transform(norm_corpus),
            vocab=np.array(cv.get_feature_names()),
            texts=norm_corpus_tokens)
        models.append(lda_model_best)
        cur_coherence_score = [i for i in cur_coherence_score if str(i)!='nan']
#         print(cur_coherence_score)
        coherence_scores.append(np.mean(cur_coherence_score))
    return models, coherence_scores

In [23]:
models, coherence_scores = topic_model_coherence_generator_c_uci(
    start, end,step, norm_corpus=np.array(data), cv=tfidf_vectorizer, cv_matrix=tfidf_vectorized)

2
4


In [24]:
coherence_scores

[-6.954995683517806, -6.954995683517806]